
# Transfer Learning

### 1. Transfer Learning

1. Take layers from a **previously trained model** (https://paperswithcode.com/dataset/imagenet).
2. **Freeze them**, so as to avoid destroying any of the information they contain during future training rounds.
3. **Add some new, trainable layers** on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
4. **Train** the new layers on your dataset.

### 2. Optional Fine-tuning

Unfreeze the entire model (or part of it), and re-train it on the new data with a *very low learning rate*. This can potentially achieve meaningful improvements, by incrementally adapting the pretrained features to the new data.

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import efficientnet

for multiclass classification with a Data Generator, it helps to organize data in this fashion:

```
./data
    ├── test
    │   ├── class1
    │   ├── class2
    │   └── class3
    └── train
        ├── class1
        ├── class2
        └── class3
```

### Read in data from local drive and define inputs and targets

In [ ]:
def get_data(base_path, classes):
    X = []
    y = []   

    for i, target in enumerate(classes):
        files = os.listdir(base_path+target)
        for file in files:
            # load the image
            img = tf.keras.utils.load_img(base_path + target + '/' + file, 
                                          target_size=(224, 224))
            # convert it to an array
            img_array = np.array(img)
            # append the array to X
            X.append(img_array)
            # append the numeric target to y
            y.append(i)

    X = np.array(X)
    y = np.array(y)

    # shuffle the data
    shuffler = np.random.permutation(len(X))
    X = X[shuffler]
    y = y[shuffler]
    return X, y

In [ ]:
classes = ['class1', 'class2', 'class3']  # replace by your own names


X, y = get_data('../data/train/', classes)
X.shape, y.shape

## Preprocess the images and targets

We need to apply the same preprocessing that was also applied to the images during training of the pre-trained model!

In [ ]:
for i, image in enumerate(X):
    X[i] = efficientnet.preprocess_input(image)
X.shape

In [ ]:
y

In [ ]:
# convert y into a one-hot-encoded matrix
y = tf.keras.utils.to_categorical(y)
y.shape

## Take layers from a previously trained model: EfficientNet

another lightweight model.

- Paper: https://arxiv.org/abs/1905.11946
- Other pre-trained models: https://keras.io/api/applications/

In [ ]:
base_model = efficientnet.EfficientNetB0(
        weights='imagenet',         # use imagnet weights
        pooling='avg',              # final flattening after convolutional layers
        include_top=False,          # we only want the base of the model
        input_shape=(224, 224, 3),  # this is the default for pre-trained models
)

In [ ]:
base_model.summary()

In [ ]:
base_model.predict(X)

#### Freeze the layers of the base model

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

#### Add new trainable layers on top of the frozen layers

In [ ]:
model=tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Dense(3, activation=tf.keras.activations.softmax))
model.summary()

#### Start the training

In [ ]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy]
)

In [ ]:
# early stopping to avoid overfitting
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [ ]:
results = model.fit(X, y, epochs=10, batch_size=64, callbacks=[callback], validation_split=0.3)

In [ ]:
plt.plot(results.history['val_loss'], label='validation loss')
plt.plot(results.history['loss'], label='training loss')
plt.legend()

#### Save the model for deployment

In [ ]:
model.save('model.h5')

#### Run some test evaluation

In [ ]:
# load in model for use
model = tf.keras.models.load_model('model.h5')
model.summary()

In [ ]:
Xtest, ytest = get_data('../data/test/', classes)
Xtest.shape, ytest.shape

In [ ]:
# preprocess the test data
for i, image in enumerate(Xtest):
    Xtest[i] = efficientnet.preprocess_input(image)
Xtest.shape

In [ ]:
ytest_pred = model.predict(Xtest)
ytest_pred.shape

In [ ]:
model.evaluate(Xtest, tf.keras.utils.to_categorical(ytest))

In [ ]:
np.round(ytest_pred, 2)

In [ ]:
# convert the class probablities into discrete categories
ytest_pred = np.argmax(ytest_pred, axis=1)
ytest_pred

In [ ]:
np.array(classes)[ytest_pred]

### Advanced Tutorial with Image Augmentation and Fine-Tuning

Read about and try out image augmentation: https://keras.io/guides/transfer_learning